Load dataset:

In [7]:
from HumanGenomeDataset.load_dataset import load_dataset

data_df = load_dataset('dna_protein_coding_sequences')
data_df.head(2)

,ID,sequence,category
0,NM_001368254.1,GCTGAGCTGAGCTGGGGCGCAGCCGCCTGTCTGCACCGGCAGCACC...,NM
1,NM_001350977.1,TCCACAACTGAAACATCCACTTCTGAACACCATGTCCTACTACAGC...,NM


Start simulation:

In [8]:
from src.simulation import ProteinSinthesisProcess
SIM_TIME = 10

In [9]:
protein_synthesis_process = ProteinSinthesisProcess(dna_sequences_df=data_df, verbose=True)
protein_synthesis_process.run(simulation_time=SIM_TIME) # run the simulation

Simulation environment initialized
Simulation started:
Time 0.0000: DNA Sequence 0 requesting to start synthesis
Time 0.0000: DNA Sequence 0 synthesize started
Time 0.0000: DNA Sequence 0 start transcription process
Time 0.0000: DNA Sequence 0 contains 6 promoters
Time 0.0000: DNA Sequence 0 (mRNA sequence 0) start transcription process
Time 0.1113: DNA Sequence 1 requesting to start synthesis
Time 0.1113: DNA Sequence 1 synthesize started
Time 0.1113: DNA Sequence 1 start transcription process
Time 0.1113: DNA Sequence 1 contains 2 promoters
Time 0.1113: DNA Sequence 1 (mRNA sequence 0) start transcription process
Time 0.7526: DNA Sequence 2 requesting to start synthesis
Time 0.7526: DNA Sequence 2 synthesize started
Time 0.7526: DNA Sequence 2 start transcription process
Time 0.7526: DNA Sequence 2 contains 1 promoters
Time 0.7526: DNA Sequence 2 (mRNA sequence 0) start transcription process
Time 0.9057: DNA Sequence 3 requesting to start synthesis
Time 0.9057: DNA Sequence 3 synthes

Results:

In [10]:
results_df = protein_synthesis_process.dna_sequences_df

In [11]:
print('Number of DNA sequence traslated:', results_df[results_df['mrna_sequences'].notna()].shape[0])
print('Number of DNA sequence not traslated:', results_df[results_df['protein_synthesized']==False].shape[0])

Number of DNA sequence traslated: 5
Number of DNA sequence not traslated: 8


In [12]:
results_df[results_df['mrna_sequences'].notna()][[
    'polypeptides_chains', 'number_of_proteins_synthesized']]

,polypeptides_chains,number_of_proteins_synthesized
13480,[NH2-RGRHASAPVVTTETSSSEPPYRVSDPLTRAPGRLRWPEMAR...,1
38095,"[NH2-QKPRWGLVYYRHKLVFRQSLY-COOH, NH2-QKPRWGLVY...",2
113258,"[NH2-V-COOH, NH2-V-COOH]",2
132315,[NH2-SEPPTRSTAEERERATGAWTGATHGSTSLTTSRTPYGTGKG...,1
179860,[NH2-DTVYPRAKPFGKGSSGELTLVHEYITPRVNTVVSGGDGRRP...,1
